# ヘッジング戦略

オプションを使ってリスクを管理するための重要な技術です。

リスクヘッジは金融市場で非常に重要な役割を果たしており、ポートフォリオの価値が急激に変動するリスクを減少させるために使用されます。

オプション取引におけるヘッジング戦略について、以下のような基本的な戦略を学ぶことができます。

1. デルタヘッジ

デルタヘッジは、最も基本的で広く使用されているヘッジ戦略の一つです。この戦略は、**オプションのデルタ（基礎資産の価格変動に対するオプション価格の感度）** を計算し、それに基づいて基礎資産を買ったり売ったりすることで、オプションポジションのリスクを相殺します。

2. デルタヘッジの基本的な考え方

デルタ (Δ) は、**オプションの価格変動が基礎資産の価格変動にどれだけ敏感であるか**を表します。デルタが1の場合、オプションの価格は基礎資産の価格変動と同じ割合で変動します。
デルタヘッジでは、オプションポジションを持っている場合、そのデルタ値に基づいて、基礎資産を売買することでリスクを減少させます。

例：デルタが0.5のコールオプションを100株購入している場合、基礎資産（株式）を50株（100株×0.5）売ることで、価格変動の影響を相殺します。

3. デルタヘッジの実践

オプションのデルタは市場の変動に応じて常に変化します。そのため、デルタヘッジを継続的に調整する必要があります（これを「ダイナミックヘッジ」と呼ぶ）。

たとえば、株価が上昇するとデルタが増加するため、追加のヘッジが必要になります。


In [2]:
import numpy as np
import scipy.stats as stats

# ブラックショールズモデルでデルタ計算
def black_scholes_delta(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        return stats.norm.cdf(d1)
    elif option_type == 'put':
        return stats.norm.cdf(d1) - 1
    else:
        raise ValueError("Option type must be 'call' or 'put'.")

# パラメータ設定
S = 100  # 現在の株価
K = 100  # 行使価格
T = 30 / 365  # 満期までの日数
r = 0.05  # 無リスク金利
sigma = 0.2  # ボラティリティ

# デルタの計算
delta = black_scholes_delta(S, K, T, r, sigma, option_type='call')
print(f"デルタ値: {delta}")

# 例: デルタヘッジに基づき基礎資産を購入
# 例えば、オプション100単位のポジションを取ると、基礎資産をデルタ分だけ購入する
position_size = 100
hedge_size = position_size * delta
print(f"ヘッジサイズ: {hedge_size}株")


デルタ値: 0.5399635456230846
ヘッジサイズ: 53.996354562308454株


1. ガンマヘッジ

ガンマヘッジは、**デルタの変化に基づくリスクを管理する戦略**です。オプションのデルタは基礎資産の価格変動に応じて変動しますが、このデルタの変動を管理するためにガンマ（デルタの変化率）を利用します。

2. ガンマの概念

ガンマ (Γ) は、オプションのデルタが基礎資産の価格変動に対してどれだけ変動するかを示します。

高いガンマを持つオプションは、基礎資産の価格変動に対して敏感であり、そのデルタは大きく変動する可能性があるため、これに対するヘッジが必要です。

3. ガンマヘッジの実践

ガンマヘッジは、デルタが不安定である状況（例えば、短期オプションや価格変動が大きい場合）で使用されます。

具体的には、オプションポジションを持つとき、ガンマをヘッジするためにさらにオプションポジションを取ることが考えられます。例えば、デルタが変化するのを見越して、別のオプションを購入してガンマリスクを管理することができます。

In [3]:
# ブラックショールズモデルでガンマ計算
def black_scholes_gamma(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return stats.norm.pdf(d1) / (S * sigma * np.sqrt(T))

# ガンマの計算
gamma = black_scholes_gamma(S, K, T, r, sigma)
print(f"ガンマ値: {gamma}")

# ガンマヘッジ
# 追加で取るべきオプションポジションを計算
additional_position = gamma * position_size**2  # ガンマヘッジにはオプションのポジションサイズの2乗に比例する
print(f"追加ポジション: {additional_position}")


ガンマ値: 0.06922764046846869
追加ポジション: 692.2764046846869


1. セータヘッジ

セータヘッジは、**オプションの価値が時間の経過とともにどのように変動するかを管理するための戦略**です。オプションの価値は、時間が経過するにつれて減少するため、これを考慮したヘッジ戦略を取ることが重要です。

2. セータの概念

セータ (Θ) は、オプションの価格が時間の経過に伴ってどれだけ減少するかを示します。
セータが負の値の場合、オプションは時間の経過とともに価値が減少し、これにより売りオプション（ショートポジション）を取ることで利益を得ることができます。

3. セータヘッジの実践

長期オプションポジションを持つ場合、時間の経過による価値減少（タイムデケイ）に対するヘッジが必要です。
例えば、ショートポジションを取ることで、タイムデケイの影響を反対に利用することができます。


In [4]:
# ブラックショールズモデルでセータ計算
def black_scholes_theta(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        theta = (-S * stats.norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * stats.norm.cdf(d2)
    elif option_type == 'put':
        theta = (-S * stats.norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * stats.norm.cdf(-d2)
    else:
        raise ValueError("Option type must be 'call' or 'put'.")
    
    return theta

# セータの計算
theta = black_scholes_theta(S, K, T, r, sigma, option_type='call')
print(f"セータ値: {theta}")

# セータヘッジ
# 時間経過に伴うオプションの価値減少リスクをヘッジするため、ポジションを調整
hedge_adjustment = -theta * position_size  # ポジション調整
print(f"セータ調整によるポジション変更: {hedge_adjustment}")


セータ値: -16.420676980838977
セータ調整によるポジション変更: 1642.0676980838975


1. ベガヘッジ

ベガヘッジは、**オプションの価値がボラティリティ（価格の変動幅）の変動に対してどれだけ感度があるかを管理する戦略**です。

2. ベガの概念

ベガ (ν) は、オプションの価格がボラティリティの変動に対してどれだけ影響を受けるかを示します。

高いボラティリティを持つ市場では、オプションの価格は高くなるため、これに対して適切にヘッジを行う必要があります。

3. ベガヘッジの実践

ボラティリティの予測が難しい場合、ベガヘッジを使ってオプションポジションのリスクを管理します。

例えば、ボラティリティが増加すると予想される場合、ロングオプションポジションを取って、ボラティリティの上昇に備えることができます。

In [5]:
# ブラックショールズモデルでベガ計算
def black_scholes_vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * np.sqrt(T) * stats.norm.pdf(d1)

# ベガの計算
vega = black_scholes_vega(S, K, T, r, sigma)
print(f"ベガ値: {vega}")

# ベガヘッジ
# ボラティリティの変動に備え、適切なオプションポジションを調整
vega_hedge = -vega * position_size  # ボラティリティの変動リスクを抑えるために調整
print(f"ベガ調整によるポジション変更: {vega_hedge}")


ベガ値: 11.379886104405811
ベガ調整によるポジション変更: -1137.9886104405812


# リバランスとダイナミックヘッジ
多くのヘッジ戦略では、ポジションを定期的にリバランスしてダイナミックヘッジを行います。市場が変動すると、オプションのデルタやガンマ、セータ、ベガなどの値も変わるため、これに応じてヘッジを調整する必要があります。

# ダイナミックヘッジの実践
継続的に市場の動きを監視し、オプションポジションを再調整してリスクを最小化します。
例えば、基礎資産の価格が大きく変動した場合、デルタヘッジを再調整してリスクを管理します。